## 데이터 전처리

### 관련 라이브러리 호출

In [ ]:
# 관련 라이브러리를 호출합니다.
import os
import chardet
import numpy as np
import pandas as pd
import joblib

### 작업 경로 확인 및 변경

In [ ]:
# 현재 작업 경로를 확인합니다.
os.getcwd()

In [ ]:
# data 폴더로 작업 경로를 변경합니다.
os.chdir(path = '../data')

In [ ]:
# 현재 작업 경로에 있는 폴더명과 파일명을 출력합니다.
os.listdir()

###  xlsx 파일을 읽고 데이터프레임 생성

In [ ]:
# 가격 xlsx 파일을 읽고 데이터프레임을 생성합니다.
%time price = pd.read_excel(io = 'APT_Price_Seoul_2021.xlsx')

In [ ]:
# price의 정보를 확인합니다.
price.info()

In [ ]:
# price의 처음 5행을 출력합니다.
price.head()

### csv 파일을 읽고 데이터프레임 생성

In [ ]:
# 가격 csv 파일명을 재사용할 수 있도록 변수에 할당합니다.
file = 'APT_Price_Seoul_2021.csv'

In [ ]:
# csv 파일을 bytes 모드로 읽습니다.
text = open(file = file, mode = 'rb').read()

In [ ]:
# csv 파일에 있는 한글의 문자 인코딩 방식을 확인합니다.
chardet.detect(text[:1000])

In [ ]:
# 가격 csv 파일을 읽고 데이터프레임을 생성합니다.
%time price = pd.read_csv(filepath_or_buffer = file, encoding = 'UTF-8')

In [ ]:
# price의 열별 자료형을 확인합니다.
price.dtypes

### csv 파일의 단점 해결

In [ ]:
# 날짜 데이터는 날짜형, 콤마를 포함하는 숫자는 정수형으로 읽습니다.
df = pd.read_csv(filepath_or_buffer = file, 
                 encoding = 'UTF-8', 
                 parse_dates = ['거래일'], 
                 thousands = ',')

In [ ]:
# df의 열별 자료형을 확인합니다.
df.dtypes

### 열 선택

In [ ]:
# 열이름을 문자열 스칼라로 지정하면 시리즈로 반환합니다.
# [참고] 열이름을 리스트로 지정하면 데이터프레임으로 반환합니다.
price['지역코드']

In [ ]:
# 2개 이상의 열을 선택하려면 반드시 리스트로 지정해야 합니다.(배열 인덱싱)
# [참고] 열이름 순서를 변경하면 위치를 바꿔서 반환합니다.
price[['아파트', '지역코드']]

In [ ]:
# 슬라이스로 연속된 열을 선택하려면 loc 인덱서를 추가합니다.(슬라이싱)
# [주의] loc 인덱서를 생략하면 에러를 반환합니다.
price.loc[:, '거래일':'거래금액']

In [ ]:
# 조건을 만족하는 열을 선택합니다.(불리언 인덱싱)
price.loc[:, price.dtypes == np.int64]

In [ ]:
# 열이름에서 '거래'를 포함하는 열을 선택합니다.(불리언 인덱싱)
price.loc[:, price.columns.str.contains(pat = '거래')]

### 열 삭제

In [ ]:
# 삭제할 열이름을 drop() 함수의 columns 매개변수에 리스트로 지정합니다.
price.drop(columns = ['지역코드'])

In [ ]:
# price의 처음 5행을 출력합니다. price는 여전히 지역코드를 포함하고 있습니다.
price.head()

In [ ]:
# 열을 삭제하고 재할당하면 데이터프레임에서 해당 열을 삭제합니다.
price = price.drop(columns = ['지역코드'])
price.head()

### 열이름 변경

In [ ]:
# 일부 열이름을 변경한 결과를 출력합니다.
# [주의] columns 매개변수를 생략하면 안됩니다!
price.rename(columns = {'시도명': '시도', '시군구': '자치구'})

In [ ]:
# price의 열이름을 출력합니다.
price.columns

In [ ]:
# 전체 열이름을 변경합니다.
# [주의] 데이터프레임의 열이름과 원소 개수가 같은 리스트를 지정해야 합니다.
price.columns = ['아파트', '시도', '자치구', '읍면동', '지번', 
                 '거래일', '전용면적', '층', '거래금액']
price.head()

### 열별 자료형 변환

In [ ]:
# 시리즈의 원소 자료형을 변환합니다.
# [참고] 거래금액 원소에 콤마가 있으므로 실수로 변환할 수 없습니다.
price['거래금액'].astype(dtype = float)

In [ ]:
# 거래금액 원소에 있는 콤마를 삭제하고 재할당합니다.
price['거래금액'] = price['거래금액'].str.replace(pat = ',', repl = '')

In [ ]:
# 시리즈의 원소 자료형을 실수형으로 변환하고 재할당합니다.
price['거래금액'] = price['거래금액'].astype(dtype = float)
price.dtypes

In [ ]:
# 데이터프레임의 열별로 자료형 변환 방법을 딕셔너리로 지정합니다.
price = price.astype(dtype = {'거래일': np.datetime64, '층': float})
price.dtypes

In [ ]:
# 정수형으로 변환할 열이름으로 리스트를 생성합니다.
cols = ['층', '거래금액']

In [ ]:
# 선택한 열의 자료형을 일괄 변환합니다.
price[cols] = price[cols].astype(dtype = int)
price.dtypes

### [참고] 문자열을 날짜형으로 변환

In [ ]:
# 날짜 기본형이 아닌 문자열 리스트로 시리즈를 생성합니다.
birth = pd.Series(data = ['2000년 1월 1일'])
birth

In [ ]:
# birth를 날짜형으로 변환하려고 하면 에러를 반환합니다.
birth.astype(dtype = np.datetime64)

In [ ]:
# to_datetime() 함수는 문자열을 날짜형으로 변환합니다.
birth = pd.to_datetime(arg = birth, format = '%Y년 %m월 %d일')
birth.iloc[0]

### [참고] 날짜 시간 데이터 연산

In [ ]:
# 현재 날짜 시간 데이터를 생성합니다.
today = pd.Timestamp.today()
today

In [ ]:
# 태어나서 현재까지 살아온 기간을 dtGap에 할당합니다.
dtGap = today - birth.iloc[0]

In [ ]:
# dtGap에서 경과일수를 출력합니다.
dtGap.days

### [참고] 날짜 분해 함수

In [ ]:
# 거래일에서 년(year)을 정수형 시리즈로 반환합니다.
price['거래일'].dt.year

In [ ]:
# 거래일에서 월(month)을 정수형 시리즈로 반환합니다.
price['거래일'].dt.month

In [ ]:
# 거래일에서 일(day)을 정수형 시리즈로 반환합니다.
price['거래일'].dt.day

In [ ]:
# 거래일에서 영문 요일을 문자형 시리즈로 반환합니다.
price['거래일'].dt.day_name()

In [ ]:
# 거래일에서 한글 요일을 문자형 시리즈로 반환합니다.
price['거래일'].dt.day_name(locale = 'ko_KR')

### 조건에 맞는 행 선택: 연속형 변수

In [ ]:
# 거래금액이 100억 이상인 행을 선택하여 df1에 할당합니다.
df1 = price[price['거래금액'] >= 1000000]
df1.head()

In [ ]:
# 거래금액이 100억 미만이고 60층 이상인 행을 선택하여 df2에 할당합니다.
df2 = price[(price['거래금액'] < 1000000) & (price['층'] >= 60)]
df2.head()

### [참고] 시리즈의 비교 연산 함수

In [ ]:
# 층이 60 초과면 True, 아니면 False인 부울형 시리즈를 반환합니다.
price['층'].gt(60).sum()

In [ ]:
# 층이 60 이상이면 True, 아니면 False인 부울형 시리즈를 반환합니다.
price['층'].ge(60).sum()

In [ ]:
# 층이 60 미만이면 True, 아니면 False인 부울형 시리즈를 반환합니다.
price['층'].lt(60).sum()

In [ ]:
# 층이 60 이하이면 True, 아니면 False인 부울형 시리즈를 반환합니다.
price['층'].le(60).sum()

In [ ]:
# 층이 60이면 True, 60이 아니면 False인 부울형 시리즈를 반환합니다.
price['층'].eq(60).sum()

In [ ]:
# 층이 60이 아니면 True, 60이면 False인 부울형 시리즈를 반환합니다.
price['층'].ne(60).sum()

In [ ]:
# 비교 연산 함수는 시리즈이므로 소괄호로 감싸지 않아도 됩니다.
price['거래금액'].lt(1000000) & price['층'].ge(60)

### 조건에 맞는 행 선택: 범주형 변수

In [ ]:
# 자치구가 '강남구'인 행을 선택합니다.
price[price['자치구'].eq('강남구')]

In [ ]:
# 자치구가 '강남구' 또는 '서초구'인 행을 선택합니다.
price[price['자치구'].eq('강남구') | price['자치구'].eq('서초구')]

In [ ]:
# isin() 함수는 시리즈 원소가 리스트에 있으면 True, 없으면 False를 반환합니다.
price[price['자치구'].isin(values = ['강남구', '서초구'])]

In [ ]:
# str.contains() 함수는 원소에 패턴이 있으면 True, 없으면 False를 반환합니다.
price[price['자치구'].str.contains(pat = '강남|서초')]

### [참고] 시리즈를 문자열로 처리하는 주요 함수

In [ ]:
# 실습용 시리즈를 생성합니다.
addr = pd.Series(data = ['서울특별시 강남구', '경기도 성남시 분당구'])

In [ ]:
# 문자열(원소)마다 지정한 패턴을 변경합니다.
addr.str.replace(pat = ' ', repl = '')

In [ ]:
# 문자열(원소)을 지정한 패턴으로 분리한 결과를 데이터프레임으로 반환합니다.
addr.str.split(pat = ' ', expand = True)

In [ ]:
# 문자열(원소)마다 지정한 패턴이 있으면 시작 인덱스를 반환합니다.
# [참고] 지정한 패턴이 없으면 -1을 반환합니다.
addr.str.find(sub = '시')

In [ ]:
# 문자열(원소)을 지정한 인덱스로 자릅니다.
addr.str.slice(start = 0, stop = 2)

In [ ]:
# 문자열(원소)마다 지정한 패턴에 해당하는 문자열을 추출합니다.
# [주의] 패턴을 소괄호로 감싸야 합니다.
addr.str.extract(pat = '([가-힣]+구)')

### 행이름으로 행 삭제

In [ ]:
# df1의 처음 5행을 출력합니다.
df1.head()

In [ ]:
# 삭제할 행이름을 drop() 함수의 index 매개변수에 리스트로 지정합니다.
# [주의] 행이름에 없는 값을 지정하면 에러를 반환합니다.
df1.drop(index = [10512, 10513])

In [ ]:
# df1의 인덱스를 출력합니다.
df1.index

In [ ]:
# 인덱스를 슬라이싱한 코드로 행이름 리스트를 대신할 수 있습니다.
df1.drop(index = df1.index[0:2])

### 행이름 초기화

In [ ]:
# 행이름을 초기화한 결과를 출력합니다.
# [참고] drop = True를 추가하지 않으면 기존 행이름을 열로 추가합니다.
df1.reset_index(drop = True)

In [ ]:
# 특정 열을 인덱스로 지정합니다.
df1 = df1.set_index(keys = '아파트')
df1.head()

In [ ]:
# 행이름을 초기화하면서 기존 행이름을 열로 추가합니다.
df1 = df1.reset_index()
df1.head()

### 결측값 처리: 단순대체

In [ ]:
# 데이터프레임의 셀 값별 결측값 여부를 반환합니다.
price.isna()

In [ ]:
# 데이터프레임의 열별 결측값 개수를 계산합니다.
price.isna().sum()

In [ ]:
# 지번이 결측값인 행을 선택합니다.
price[price['지번'].isna()]

In [ ]:
# 지번이 결측값인 행에서 결측값을 빈 문자열로 대체한 결과를 반환합니다.
price[price['지번'].isna()].fillna(value = '')

### [참고] 결측값을 원본 데이터로 채워넣기

In [ ]:
# price를 깊은 복사한 imsi를 생성합니다.
imsi = price.copy()

In [ ]:
# 지번이 결측인 아파트별로 원본 데이터를 찾아서 채워넣습니다.
imsi.loc[imsi['아파트'].eq('서초포레스타2단지'), '지번'] = '384'
imsi.loc[imsi['아파트'].eq('힐스테이트 서초 젠트리스'), '지번'] = '557'

In [ ]:
# imsi의 열별 결측값 개수를 계산합니다.
imsi.isna().sum()

### [참고] 결측값을 이전 셀 값으로 채우기

In [ ]:
# 현재 작업 경로에 있는 폴더명과 파일명을 출력합니다.
os.listdir()

In [ ]:
# 법정동별 평균 거래금액 xlsx 파일을 읽고 데이터프레임을 생성합니다.
dong = pd.read_excel(io = 'APT_Mean_Price_Dong_2021.xlsx')

In [ ]:
# dong을 출력합니다.
# [참고] xlsx에서 병합한 셀은 처음 값만 제대로 읽고 나머지는 결측값으로 대체합니다.
dong

In [ ]:
# 자치구에 있는 결측값을 이전 셀 값으로 채웁니다.
# [참고] method 매개변수에 'bfill'을 지정하면 결측값을 이후 셀 값으로 채웁니다.
dong.fillna(method = 'ffill')

### 결측값 처리: 행 삭제

In [ ]:
# 데이터프레임에서 결측값이 있는 모든 행을 삭제한 결과를 반환합니다.
price.dropna()

In [ ]:
# 지번에서 결측값이 없는 행을 선택한 결과를 반환합니다.
price[~price['지번'].isna()]

In [ ]:
# 지번에서 결측값이 없는 행을 선택한 결과를 반환합니다.
price[price['지번'].notna()]

In [ ]:
# 지번에서 결측값이 아닌 행을 선택하고 price에 재할당합니다.
price = price[price['지번'].notna()]

In [ ]:
# price의 행 개수를 확인합니다.
price.shape[0]

### 파생변수 생성 : 연속형 변수

In [ ]:
# 거래금액을 전용면적으로 나누고 3.3을 곱한 평당금액을 생성합니다.
price['평당금액'] = price['거래금액'] / price['전용면적'] * 3.3
price.head()

In [ ]:
# 거래금액을 10000으로 나누면 단위가 만원에서 억원으로 바뀝니다.
price['거래금액'] = price['거래금액'] / 10000

In [ ]:
# pandas 옵션에서 실수를 출력하는 소수점 자리수를 3으로 설정합니다.
pd.options.display.precision = 3
price.head()

### 파생변수 생성 : 범주형 변수

In [ ]:
# 평당금액이 5000 이상이면 True, 아니면 False인 원소를 갖는 locs를 생성합니다.
# [참고] locs는 부울형 시리즈입니다.
locs = price['평당금액'].ge(5000)
locs.head()

In [ ]:
# 평당금액을 '5천 이상', '5천 미만'으로 구분한 금액구분을 생성합니다.
# [참고] np.where() 함수는 조건이 True일 때 두 번째 인수, False일 때 세 번째 인수를 반환합니다.
price['금액구분'] = np.where(locs, '5천 이상', '5천 미만')
price.head()

### [참고] pandas에서 추천하는 코딩 방식

In [ ]:
# price를 깊은 복사한 imsi를 다시 생성합니다.
imsi = price.copy()

In [ ]:
# 평당금액이 5000 이상인 행의 새 변수(금액구분2)에 '5천 이상'을 할당합니다.
imsi.loc[imsi['평당금액'].ge(5000), '금액구분2'] = '5천 이상'
imsi.head()

In [ ]:
# 평당금액이 5000 미만인 행의 새 변수(금액구분2)에 '5천 미만'을 할당합니다.
imsi.loc[imsi['평당금액'].lt(5000), '금액구분2'] = '5천 미만'
imsi.head()

### [참고] 구간화 함수

In [ ]:
# 연속형 변수를 세 개 이상으로 분리할 때 np.where() 함수를 중첩합니다.
np.where(price['평당금액'].ge(10000), 
         '1억 이상', 
         np.where(price['평당금액'].ge(5000), 
                  '5천 이상', 
                  '5천 미만'))

In [ ]:
# 연속형 변수를 세 개 이상으로 분리할 때 np.select() 함수를 사용합니다.
np.select(condlist = [price['평당금액'].ge(10000),
                      price['평당금액'].ge(5000),
                      price['평당금액'].lt(5000)], 
          choicelist = ['1억 이상', '5천 이상', '5천 미만'])

### 파생변수 생성: 문자형 변수 결합

In [ ]:
# 여러 문자형 변수를 + 연산자로 결합합니다.
price['주소'] = price['시도'] + ' ' + price['자치구'] + ' ' + \
                price['읍면동'] + ' ' + price['지번']

In [ ]:
# 여러 문자형 열이름으로 리스트를 생성합니다.
cols = ['시도', '자치구', '읍면동', '지번']

In [ ]:
# 데이터프레임의 행(시리즈)별로 문자열을 결합하는 함수를 반복 실행합니다.
price[cols].apply(func = lambda x: ' '.join(x), axis = 1)

### [참고] 같은 함수 반복 실행 실습

In [ ]:
# 아파트(시리즈)의 원소(문자열)별 글자수를 반환합니다.
price['아파트'].map(arg = len)

In [ ]:
# 데이터프레임의 열(시리즈)별 원소 개수를 반환합니다.
price[cols].apply(func = len, axis = 0)

In [ ]:
# 데이터프레임의 행(시리즈)별 원소 개수를 반환합니다.
price[cols].apply(func = len, axis = 1)

In [ ]:
# 데이터프레임의 셀 값(문자열)별 글자수를 반환합니다.
price[cols].applymap(func = len)

### 데이터프레임 정렬

In [ ]:
# 층을 오름차순 정렬합니다.
# [참고] ascending 매개변수에 전달하는 인수의 기본값은 True입니다.
price['층'].sort_values()

In [ ]:
# 층을 내림차순 정렬합니다.
price['층'].sort_values(ascending = False)

In [ ]:
# price를 층으로 오름차순 정렬합니다.
price.sort_values(by = ['층'])

In [ ]:
# price를 층으로 내림차순 정렬합니다.
price.sort_values(by = ['층'], ascending = False)

In [ ]:
# price를 층과 거래금액으로 내림차순 정렬합니다.
price.sort_values(by = ['층', '거래금액'], ascending = False)

In [ ]:
# price를 층은 내림차순, 거래금액은 오름차순 정렬합니다.
price.sort_values(by = ['층', '거래금액'], ascending = [False, True])

### 집계함수로 데이터 요약

In [ ]:
# 거래금액에서 결측값을 제외한 빈도수를 반환합니다.
price['거래금액'].count()

In [ ]:
# 거래금액의 합계를 반환합니다.
price['거래금액'].sum()

In [ ]:
# 거래금액의 평균을 반환합니다.
price['거래금액'].mean()

In [ ]:
# 거래금액의 표준편차를 반환합니다.
price['거래금액'].std()

In [ ]:
# 거래금액의 최솟값을 반환합니다.
price['거래금액'].min()

In [ ]:
# 거래금액의 최댓값을 반환합니다.
price['거래금액'].max()

### 범주별 집계함수로 데이터 요약

In [ ]:
# 자치구별 거래금액에서 결측값을 제외한 빈도수를 반환합니다.
price.groupby(by = ['자치구'])['거래금액'].count()

In [ ]:
# 자치구별 거래금액의 합계를 반환합니다.
price.groupby(by = ['자치구'])['거래금액'].sum()

In [ ]:
# 자치구별 거래금액의 평균을 반환합니다.
price.groupby(by = ['자치구'])['거래금액'].mean()

In [ ]:
# 자치구별 거래금액의 표준편차를 반환합니다.
price.groupby(by = ['자치구'])['거래금액'].std()

In [ ]:
# 자치구별 거래금액의 최솟값을 반환합니다.
price.groupby(by = ['자치구'])['거래금액'].min()

In [ ]:
# 자치구별 거래금액의 최댓값을 반환합니다.
price.groupby(by = ['자치구'])['거래금액'].max()

### 범주형 변수의 빈도수/상대도수 확인

In [ ]:
# 자치구에서 중복을 제거한 원소를 반환합니다.
price['자치구'].unique()

In [ ]:
# 자치구에서 중복을 제거한 원소 개수를 반환합니다.
price['자치구'].nunique()

In [ ]:
# 자치구별 빈도수를 내림차순 정렬한 결과를 반환합니다.
price['자치구'].value_counts()

In [ ]:
# 자치구별 빈도수를 오름차순 정렬한 결과를 반환합니다.
price['자치구'].value_counts(ascending = True)

In [ ]:
# 자치구별 빈도수를 시리즈 인덱스로 오름차순 정렬한 결과를 반환합니다.
price['자치구'].value_counts().sort_index()

In [ ]:
# 자치구별 상대도수를 내림차순 정렬한 결과를 반환합니다.
price['자치구'].value_counts(normalize = True)

### Long type 데이터프레임 생성

In [ ]:
# 두 범주형 변수명으로 리스트를 생성합니다.
cols = ['자치구', '금액구분']

In [ ]:
# 두 범주형 변수의 빈도수로 Long type 데이터프레임을 생성합니다.
elong = price.groupby(by = cols)[['평당금액']].count()
elong.head()

In [ ]:
# elong의 행이름을 초기화하고 기존 행이름을 열로 추가합니다.
elong = elong.reset_index()
elong.head()

In [ ]:
# elong의 일부 열이름을 변경합니다.
elong = elong.rename(columns = {'평당금액': '매매건수'})

### Long type을 Wide type으로 변환

In [ ]:
# Long type을 Wide type으로 변환합니다.
widen = elong.pivot(index = '자치구', columns = '금액구분', values = '매매건수')
widen.head()

In [ ]:
# widen의 행이름을 출력합니다.
# [참고] 인덱스(행이름)에 name 속성이 있고, 속성값은 '자치구'입니다.
widen.index

In [ ]:
# 인덱스(행이름)의 name을 출력합니다.
widen.index.name

### widen 행이름 초기화

In [ ]:
# widen의 행이름을 초기화하고 기존 행이름을 열로 추가합니다.
widen = widen.reset_index()
widen.head()

In [ ]:
# widen의 열이름을 출력합니다.
# [참고] 컬럼명(열이름)에 name 속성이 있고, 속성값은 '금액구분'입니다.
widen.columns

In [ ]:
# 컬럼명(열이름)의 name을 출력합니다.
widen.columns.name

In [ ]:
# 컬럼명(열이름)의 name에 빈 문자열을 할당합니다.
widen.columns.name = ''
widen.head()

### Wide type을 Long type으로 변환

In [ ]:
# Wide type을 Long type으로 변환합니다.
widen.melt(id_vars = '자치구', 
           value_vars = ['5천 미만', '5천 이상'], 
           var_name = '금액종류',
           value_name = '거래건수') \
     .sort_values(by = ['자치구', '금액종류']) \
     .reset_index(drop = True) \
     .head()

### 피벗 테이블 생성

In [ ]:
# 두 범주형 변수로 연속형 변수를 요약한 피벗 테이블을 생성합니다.
pd.pivot_table(data = price, 
               values = '평당금액', 
               index = '자치구', 
               columns = '금액구분', 
               aggfunc = np.mean)

### 교차 테이블 생성

In [ ]:
# 두 범주형 변수의 빈도수/상대도수를 원소로 갖는 교차 테이블을 생성합니다.
pd.crosstab(index = price['자치구'],
            columns = price['금액구분'], 
            normalize = 'index',
            margins = True, 
            margins_name = '합계')

### 데이터프레임 결합

In [ ]:
# 두 데이터프레임의 열이름이 순서까지 정확하게 같은지 확인합니다.
# [참고] 순서만 달라도 False를 반환합니다.
df1.columns.equals(other = df2.columns)

In [ ]:
# 열이름이 같은 두 데이터프레임을 행(세로) 방향으로 결합합니다.
# [참고] 두 데이터프레임의 기존 행이름을 유지합니다.
pd.concat(objs = [df1, df2])

In [ ]:
# 두 데이터프레임을 세로로 결합하고 행이름을 초기화합니다.
pd.concat(objs = [df1, df2], ignore_index = True)

In [ ]:
# df2의 일부 열이름을 변경합니다.
df2 = df2.rename(columns = {'아파트': '아파트명'})

In [ ]:
# 두 데이터프레임의 열이름이 순서까지 정확하게 같은지 확인합니다.
df1.columns.equals(other = df2.columns)

In [ ]:
# 열이름이 다른 두 데이터프레임을 행(세로) 방향으로 결합합니다.
pd.concat(objs = [df1, df2], ignore_index = True)

In [ ]:
# 두 데이터프레임을 열(가로) 방향으로 결합합니다.
pd.concat(objs = [df1, df2], axis = 1)

In [ ]:
# df2의 행이름을 초기화합니다.
df2 = df2.reset_index(drop = True)

In [ ]:
# 행이름을 초기화한 두 데이터프레임을 열(가로) 방향으로 결합합니다.
pd.concat(objs = [df1, df2], axis = 1)

### 병합 데이터셋 준비

In [ ]:
# 현재 작업 경로에 있는 폴더명과 파일명을 출력합니다.
os.listdir()

In [ ]:
# 상세정보 xlsx 파일을 읽고 데이터프레임을 생성합니다.
detail = pd.read_excel(io = 'Naver_APT_Detail_Seoul.xlsx')

In [ ]:
# detail의 정보를 확인합니다.
detail.info()

In [ ]:
# detail의 처음 5행을 출력합니다.
detail.head()

### 외래키 확인

In [ ]:
# 두 데이터프레임의 외래키에서 일치하는 원소 개수를 확인합니다.
len(set(price['아파트']) & set(detail['아파트명']))

In [ ]:
# 두 데이터프레임의 외래키에서 일치하는 원소 개수를 확인합니다.
len(set(price['주소']) & set(detail['지번주소']))

In [ ]:
# price의 주소를 출력합니다.
price['주소'].head()

In [ ]:
# detail의 지번주소를 출력합니다.
detail['지번주소'].head()

### 외래키 전처리

In [ ]:
# price의 주소에서 '특별'을 삭제합니다.
price['주소'] = price['주소'].str.replace(pat = '특별', repl = '')

In [ ]:
# 외래키에서 일치하는 원소 개수를 다시 확인합니다.
len(set(price['주소']) & set(detail['지번주소']))

### [참고] 비복원추출 및 시드 고정

In [ ]:
# 이번 주 로또 번호를 출력합니다. 마음에 들 때까지 여러 번 반복해보세요.
np.random.choice(a = range(1, 46), size = 6, replace = False)

In [ ]:
# 시드를 고정하면 항상 재현 가능한 결과를 얻습니다.
np.random.seed(seed = 1)
np.random.choice(a = range(1, 46), size = 6, replace = False)

### [참고] 복원추출

In [ ]:
# 시드를 고정합니다.
np.random.seed(seed = 2)

In [ ]:
# 1~5의 정수에서 3개를 복원추출합니다.
nums = np.random.choice(a = range(1, 6), size = 5)

In [ ]:
# nums를 시리즈로 변환합니다.
nums = pd.Series(data = nums)
nums

### [참고] 중복 원소 확인 함수

In [ ]:
# 순방향으로 원소 중복 여부를 True 또는 False로 반환합니다.
nums.duplicated()

In [ ]:
# 역방향으로 원소 중복 여부를 True 또는 False로 반환합니다.
nums.duplicated(keep = 'last')

In [ ]:
# 모든 중복 원소를 True로 반환합니다.(탐색 방향과 상관 없음)
nums.duplicated(keep = False)

In [ ]:
# 중복 원소 중 맨 처음 원소만 선택합니다.
nums[nums.duplicated()]

In [ ]:
# 중복 원소 중 맨 마지막 원소만 선택합니다.
nums[nums.duplicated(keep = 'last')]

In [ ]:
# 중복 원소를 모두 선택합니다.
nums[nums.duplicated(keep = False)]

### 데이터프레임 중복 확인 및 제거

In [ ]:
# detail의 지번주소가 중복이면 True, 아니면 False인 시리즈를 생성합니다.
dup = detail['지번주소'].duplicated(keep = False)

In [ ]:
# detail에서 dup이 True인 행을 선택하고 지번주소로 오름차순 정렬합니다.
detail[dup].sort_values(by = ['지번주소'])

In [ ]:
# detail의 지번주소에서 순방향으로 중복인 행을 제거하고 detail에 재할당합니다.
# [참고] ~ 연산자는 진리값을 반전합니다.
detail = detail[~detail['지번주소'].duplicated()]

In [ ]:
# detail의 행 개수를 확인합니다. 행 개수가 감소했습니다.(9668 -> 9640)
detail.shape[0]

### 데이터프레임 병합

In [ ]:
# price와 detail에서 일치하는 열이름을 확인합니다.
set(price.columns) & set(detail.columns)

In [ ]:
# 두 데이터프레임으로 내부 병합을 실행합니다.
pd.merge(left = price, 
         right = detail, 
         how = 'inner', 
         left_on = '주소', 
         right_on = '지번주소')

In [ ]:
# detail의 외래키 이름을 '주소'로 변경합니다.
detail = detail.rename(columns = {'지번주소': '주소'})

In [ ]:
# price와 detail에서 일치하는 열이름을 확인합니다.
set(price.columns) & set(detail.columns)

In [ ]:
# 외래키 이름이 같으면 on 매개변수를 사용합니다.
apt = pd.merge(left = price, right = detail, how = 'inner', on = '주소')

In [ ]:
# apt의 정보를 확인합니다.
apt.info()

### 외부 파일로 저장

In [ ]:
# apt를 xlsx 파일로 저장합니다.
%time apt.to_excel(excel_writer = 'APT_List_Seoul_2021.xlsx', index = None)

In [ ]:
# apt를 csv 파일로 저장합니다.
%time apt.to_csv(path_or_buf = 'APT_List_Seoul_2021.csv', index = None)

In [ ]:
# apt를 z 파일로 저장합니다.
%time joblib.dump(value = apt, filename = 'APT_List_Seoul_2021.z')

## End of Document